# OpenAI 매개변수

# 개요
OpenAI 모델에 요청을 보낼 때 여러 매개변수를 사용하여 모델의 동작과 출력을 제어할 수 있습니다. \
이러한 매개변수를 이해하면 텍스트 생성, 질문 응답 또는 기타 사용 사례에 맞게 응답을 세부 조정할 수 있습니다.

더 자세한 예제는 공식 문서를 참조하세요: [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
SEED = 123

# 매개변수: max_tokens
**설명**: 생성할 응답의 최대 토큰 수를 설정합니다. \
**기본값**: 16 \
**예제**: max_tokens=50

프롬프트의 토큰 수와 max_tokens의 합은 모델의 컨텍스트 길이를 초과할 수 없습니다. \
gpt-4o-mini의 경우 16,384 tokens이며 모델에 따라 다릅니다. 

In [3]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "최고의 반려동물은 "}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

tokens = [32, 64, 120, 200]
for token in tokens:
    print(f"Max Tokens: {token}\n")
    print(call_openai_with_max_tokens(token))
    print("\n" + "-"*80 + "\n")

Max Tokens: 32

“최고의 반려동물”은 사람마다 다를 수 있습니다. 각자의 생활 패턴, 환경, 취향, 알레르기

--------------------------------------------------------------------------------

Max Tokens: 64

"최고의 반려동물"은 사람마다 다르게 느낄 수 있어요. 라이프스타일, 거주 환경, 시간, 취향 등에 따라 각기 다른 동물이 최고의 반려동물이 될 수 있답니다.

예를 들면:

- **강아지**:

--------------------------------------------------------------------------------

Max Tokens: 120

최고의 반려동물은 사람마다 다르게 생각할 수 있습니다. 각각의 반려동물은 고유한 장점과 특징이 있기 때문입니다. 몇 가지 반려동물의 특징을 예로 들어드릴게요:

1. **강아지**  
   - 특징: 충성심이 강하고, 다양한 품종이 있어 목적에 따라 선택 가능
   - 장점: 적극적인 교감, 산책 등 활동적인 삶 가능
   - 고려 사항: 충분한 시간과 관심 필요

2. **고양이**

--------------------------------------------------------------------------------

Max Tokens: 200

“최고의 반려동물”은 사람마다, 상황마다 다릅니다. 어떤 반려동물이 최고의 선택이 될지는 다음과 같은 요소에 따라 달라집니다:

1. 라이프스타일  
- 활동적인 사람이라면 산책이 필요한 강아지가 잘 맞을 수 있습니다.  
- 집에 오래 머무는 사람이라면 고양이나 소형 조류, 물고기 등이 어울릴 수 있습니다.

2. 공간  
- 넓은 집이나 마당이 있으면 대형견도 좋습니다.  
- 아파트나 작은 공간에는 소형견이나 고양이, 작은 동물들이 적합합니다.

3. 시간  
- 반려동물에게 할애할 시간이 많다면 손이 많이 가는 동물을 키워도 됩니다.  

# 매개변수: temperature

**설명**: 출력의 무작위성을 제어합니다. 낮은 값은 출력을 더 결정론적으로 만들고, 높은 값은 무작위성을 증가시킵니다. \
모델이 다음 토큰을 선택할 때, 여러 후보 중에서
- 높은 확률 토큰을 쓸지
- 조금 낮은 확률 토큰까지 선택할 여지를 줄지

**값 범위**: 0에서 2 \
**기본값**: 1 \
**예제**: temperature=0.7

높은 값은 모델이 더 창의적인 출력을 생성하도록 합니다. \
창의적인 응용 프로그램에는 0.9를, 명확한 답변이 필요한 경우에는 0(최대 확률 샘플링)을 시도하세요.

---
**참고**: 일반적으로 이 매개변수 또는 top_p를 조정하되 둘 다 동시에 조정하지 않는 것을 권장합니다.


In [4]:
def call_openai(num_times, prompt, temperature=0.8, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [5]:
# Without seed and temperature, the response is different each time
call_openai(10, '최고의 반려동물은 ')

"최고의 반려동물"은 사람마다 다르게 생각할 수 있습니다. 개인의 생활 방식, 성격, 환경, 반려동물에게 줄 수 있는 시간과 노력 등에 따라 가장 적합한 반려동물이 달라질 수 있습니다. 아래에 몇 가지
최고의 반려동물은 사람마다 다를 수 있습니다. 개인의 라이프스타일, 성격, 주거 환경, 알레르기 유무 등에 따라 최적의 반려동물이 달라질 수 있지요. 대표적인 반려동물과 그
"최고의 반려동물"은 사람마다 다르게 느낄 수 있습니다. 각자의 생활환경, 성격, 취향에 따라 최고의 반려동물은 달라질 수 있기 때문입니다. 몇 가지 예를 들어볼게요:

1. 강아지
“최고의 반려동물”이라는 질문은 사람마다 답이 다를 수 있어요. 각자의 생활 방식, 성격, 환경, 그리고 반려동물에게 바라는 점이 다르기 때문이죠. 몇 가지 예를 들어볼게요:

1.
최고의 반려동물은 사람마다 다를 수 있습니다. 각자 생활 환경, 성격, 취향, 가족 구성원 등에 따라 적합한 반려동물이 달라지기 때문입니다. 몇 가지 대표적인 반려동물과 특징을 소개해드릴
최고의 반려동물에 대한 답은 사람마다 다를 수 있습니다. 각자의 라이프스타일, 성격, 거주 환경, 취향 등에 따라 "최고"라고 느끼는 반려동물이 다르기 때문입니다. 아래는 다양한 반려동
최고의 반려동물은 사람마다 다르게 생각할 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 적합한 반려동물이 다르기 때문입니다. 아래에 몇 가지 반려동물을 예시로 들어드릴게요.

1
최고의 반려동물이 무엇인지에 대한 답은 사람마다 다를 수 있습니다. 각자의 생활 방식, 성격, 환경, 취향에 따라 어울리는 반려동물이 다르기 때문입니다. 아래에 대표적인 반려동물 종류와 그 특징을
"최고의 반려동물"은 사람마다 다를 수 있습니다. 각자의 생활환경, 성격, 취향에 따라 적합한 반려동물이 다르기 때문입니다. 아래에는 대표적인 반려동물들과 그 장단점을 소개해드릴게
최고의 반려동물은 사람마다 다를 수 있습니다. 각자의 생활환경, 성격, 취향, 시간적 여유 등에 따라 최적의 반

In [6]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, '최고의 반려동물은 ', temperature = 0, use_seed=True)

"최고의 반려동물"은 사람마다 다르게 생각할 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 몇 가지 예를 들어볼게요:

1. **강아지
"최고의 반려동물"은 사람마다 다르게 생각할 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 몇 가지 예를 들어볼게요:

1. **강아지
"최고의 반려동물"은 사람마다 다를 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 아래에 몇 가지 대표적인 반려동물과 그 특징을 소개해드
"최고의 반려동물"은 사람마다 다르게 생각할 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 아래는 대표적인 반려동물과 그 특징입니다:

1.
"최고의 반려동물"은 사람마다 다를 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 아래에 몇 가지 대표적인 반려동물과 그 특징을 소개해드
최고의 반려동물은 사람마다 다를 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 몇 가지 예를 들어볼게요:

1. **강아지**  
사교
"최고의 반려동물"은 사람마다 다를 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 아래는 대표적인 반려동물과 그 특징입니다:

1. 강아
"최고의 반려동물"은 사람마다 다르게 생각할 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 아래는 대표적인 반려동물과 그 특징입니다:

1.
"최고의 반려동물"은 사람마다 다를 수 있습니다. 각자의 생활 방식, 성격, 환경에 따라 가장 잘 맞는 반려동물이 달라지기 때문입니다. 아래에 몇 가지 대표적인 반려동물과 그 특징을 소개해드
"최고의 반려동물"은 사람마다 다를 수 있습니다. 각자의 생활 방

# 매개변수: n
**설명**: 각 프롬프트에 대해 생성할 응답의 수를 지정합니다. \
**기본값**: 1 \
**예제**: n = 3 

---
**참고**: 이 매개변수는 여러 응답을 생성하므로 토큰 할당량을 빠르게 소모할 수 있습니다. 신중하게 사용하고 max_tokens 및 stop 설정이 적절한지 확인하세요.

In [7]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "최고의 반려동물은"}],
                max_tokens=60,
                n=2
        )

for index, c in enumerate(response.choices):
    print(index, c.message.content)
    print("\n" + "-"*80 + "\n")

0 "최고의 반려동물"은 사람마다 다를 수 있습니다. 라이프스타일, 집의 크기, 시간적 여유, 알레르기 여부 등 여러 가지 조건에 따라 최적의 반려동물이 달라지기 때문입니다. 아래에 상황

--------------------------------------------------------------------------------

1 "최고의 반려동물"은 사람마다 다를 수 있습니다. 개인의 라이프스타일, 성격, 생활 환경, 시간적 여유, 알레르기, 관심사 등에 따라 최고의 반려동물이 달라집니다. 아래에 대표적인 반려

--------------------------------------------------------------------------------



# 매개변수: presence_penalty
**설명**: 텍스트에 이미 나타난 토큰을 기반으로 새 토큰에 페널티를 부여하여 모델이 새로운 토큰을 사용하도록 유도합니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: presence_penalty=0.5

**특징**  
- 단어가 *한 번이라도 등장*했으면 그 이후 사용 확률을 낮춤  
- "존재 여부" 기반이므로 반복 횟수와는 무관  
- 표현을 살짝 다양하게 하고 싶을 때 적합  
- 자연스러움을 크게 해치지 않으면서 중복을 줄이는 용도

**주의 사항**  
- 코드 생성·SQL·JSON·Tool Call처럼 **반복이 필수인 출력에서는 사용 금지**  
- 주제어가 반복될 수밖에 없는 설명형 프롬프트에서는 효과가 제한적  
- 값이 너무 높으면 문맥과 자연스러움이 손상될 수 있음

---

### presence_penalty vs frequency_penalty 비교

| 특징 | presence_penalty | frequency_penalty |
|------|------------------|-------------------|
| **기준** | 존재 여부 (0/1) | 반복 횟수 (누적) |
| **강도** | 약함 | 강함 |
| **효과** | 미묘한 변화 | 확실한 억제 |
| **용도** | 자연스러운 다양성 | 반복 패턴 제거 |
| **예시** | 일반 글쓰기 | 창의적 콘텐츠 |


In [8]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
            {"role": "user", "content": "'고양이'에 대한 10문장을 작성해줘. 단, 가능한 자연스럽게 작성해줘."}
        ],
                    max_tokens=400,
                    presence_penalty=presence_penalty, 
                    
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

1. 고양이는 매우 유연한 몸을 가지고 있어 높은 곳도 쉽게 오를 수 있다.  
2. 고양이는 혼자 있는 것을 좋아하지만, 사람과 교감할 때도 많다.  
3. 고양이의 털은 종류와 색이 다양해서 다양한 모습이 있다.  
4. 고양이는 야간에 활동하는 습성이 있어 밤에도 잘 보인다.  
5. 고양이는 깨끗한 환경을 좋아해서 자주 스스로를 그루밍한다.  
6. 고양이는 특히 장난감을 가지고 노는 것을 좋아한다.  
7. 고양이가 기분이 좋을 때는 가르릉 소리를 낸다.  
8. 고양이는 사냥 본능이 강해서 작은 움직임에도 민감하게 반응한다.  
9. 고양이는 집안에서 키우기 좋은 반려동물로 인기가 많다.  
10. 고양이와 함께 있으면 편안하고 행복한 기분을 느낄 수 있다.

--------------------------------------------------------------------------------

Presence Penalty: 0.5

1. 고양이는 아주 유연한 몸을 가지고 있어서 높은 곳도 쉽게 올라갈 수 있다.  
2. 대부분의 고양이는 혼자 있는 시간을 좋아하지만, 사람과 교감하는 것도 즐긴다.  
3. 고양이는 털을 깨끗하게 하기 위해 스스로 그루밍을 자주 한다.  
4. 고양이의 눈은 밤에도 잘 보이도록 특별하게 진화했다.  
5. 고양이는 자신이 관심 있는 것에는 호기심을 많이 보인다.  
6. 많은 사람들이 고양이의 귀여운 모습을 좋아해서 반려동물로 키운다.  
7. 고양이는 걷거나 뛸 때 매우 조용하게 움직인다.  
8. 고양이가 내는 그르렁거리는 소리는 평온함을 느낄 때 나타나는 특징이다.  
9. 고양이는 사냥 본능이 강해서 작은 장난감에도 빠르게 반응한다.  
10. 고양이는 사람의 감정을 잘 파악해서 위로를 건네기도 한다.

--------------------------------------------------------------------------------

Presenc

# 매개변수: frequency_penalty
**설명**: 텍스트에 이미 나타난 빈도를 기반으로 새 토큰에 페널티를 부여하여 동일한 줄을 반복할 가능성을 줄입니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: frequency_penalty=0.5

**특징**  
- 동일 단어가 반복될수록 페널티 강도가 증가  
- presence_penalty보다 훨씬 강한 억제 효과  
- 반복되는 문장·패턴을 확실하게 줄이고 싶을 때 적합  
- 창의적 글쓰기, 마케팅 문구, 스토리텔링에 유용

**주의 사항**  
- 변수명·키 이름·구문 반복이 중요한 **코드/SQL/JSON 생성에는 절대 사용하면 안 됨**  
- 높은 값일수록 다양성은 증가하지만 문맥 일관성은 낮아질 수 있음  
- 정보 전달이나 기술적 설명 같은 정확성이 필요한 응답에는 추천하지 않음

In [9]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "'고양이'에 대한 10문장을 작성해줘. 단, 가능한 자연스럽게 작성해줘."}
        ],
        max_tokens=400,
        frequency_penalty=frequency_penalty
    )
    return response.choices[0].message.content


# Generate with different frequency_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Frequency Penalty: 0

1. 고양이는 부드러운 털을 가지고 있어 만질 때 기분이 좋다.  
2. 대부분의 고양이는 독립적인 성격을 가지고 있다.  
3. 고양이는 낮잠을 자는 것을 아주 좋아한다.  
4. 귀엽게 울면서 사람의 관심을 끌기도 한다.  
5. 창가에 앉아 바깥 풍경을 바라보는 것을 즐긴다.  
6. 고양이는 작은 움직임에도 민감하게 반응한다.  
7. 손에 장난감을 들고 흔들면 뛰어오르며 놀기도 한다.  
8. 가끔씩 집안을 뛰어다니면서 에너지를 발산한다.  
9. 고양이는 스스로 그루밍하면서 털을 깨끗하게 관리한다.  
10. 사람 옆에 다가와서 조용히 앉아 있을 때가 많다.

--------------------------------------------------------------------------------

Frequency Penalty: 0.5

1. 고양이는 매우 독립적인 성격을 가지고 있어요.  
2. 집에서 키우는 고양이는 보통 하루에 여러 번 낮잠을 자곤 해요.  
3. 고양이는 깨끗한 곳을 좋아하기 때문에 스스로 몸을 자주 핥아서 청결을 유지해요.  
4. 밤에 활동적인 모습을 보일 때가 많아요.  
5. 많은 사람들이 고양이의 귀여운 외모와 행동 때문에 반려동물로 많이 키워요.  
6. 고양이는 평소 조용하지만 호기심이 많아서 새로운 것을 발견하면 바로 다가가 살펴봐요.  
7. 자신의 마음에 들면 애교를 부리기도 하지만, 싫으면 혼자 있고 싶어해요.  
8. 고양이는 먹이를 챙겨주는 사람에게 애정 표현을 하기도 해요.  
9. 다양한 색깔과 무늬를 가진 고양이들이 있어 구경하는 재미도 쏠쏠해요.  
10. 밝고 따뜻한 곳에서 햇볕을 쬐는 것을 매우 좋아하는 동물이에요.

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

네, 아래에 "고양이"에 대한 자연스러운 

### 탐색할 사용 사례
1. **응답 비교**  
   여러 응답을 생성하여 사용 사례에 가장 적합한 결과를 선택하세요.

2. **다양성 증가**  
   창의적 응용 프로그램에서 다양한 표현을 얻기 위해 여러 응답을 생성하세요.

3. **강건성 향상**  
   여러 응답을 생성하여 일관성과 정확성을 비교·검증하세요.

---

### 모범 사례
1. **프롬프트 길이 최적화**  
   프롬프트는 간결하지만 충분한 정보를 담도록 작성하세요.

2. **Temperature 및 Top_p 조정**  
   결정론적 vs. 창의적 응답의 균형을 위해 온도를 적절히 조정하세요.

3. **토큰 사용량 모니터링**  
   `max_tokens`를 적절히 설정하여 비용과 응답 길이를 관리하세요.

4. **중지 시퀀스 사용**  
   모델이 텍스트 생성을 중단해야 하는 지점을 정의해 출력을 안정적으로 제어하세요.

5. **여러 응답 생성**  
   `n` 매개변수를 사용하여 여러 응답을 생성하고 필요한 응답을 선택하세요.